# TODO
    - address Ne = 50    
    - break out the fuctions into a separate file.
    - incorporate seeds into the two simulation steps as well as the sampling steps.
    - batch simulation steps 
    - figure out ancestral ne, and what scaling the mutation rate means in practice
    - figure out how to simulate unlinked loci (just run msprime many, many times??)
    - check the controlled mating is doing the expected
    - batch the analysis


In [3]:
import numpy as np
import pandas as pd
import itertools
#import scipy.spatial.distance
#import matplotlib.pyplot as plt
#import seaborn as sns
import collections
import glob
import random
from random import shuffle
import os.path

import msprime
#import allel

#import simuPOP
#import simuPOP.utils

from pylab import rcParams
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline
#sns.set_style('white')

In [7]:
!jupyter nbconvert --to script simNe_functions_simulate.ipynb
import simNe_functions_simulate as simNe_sim

[NbConvertApp] Converting notebook simNe_functions_simulate.ipynb to script
[NbConvertApp] Writing 14168 bytes to simNe_functions_simulate.py
simuPOP Version 1.1.7.1 : Copyright (c) 2004-2016 Bo Peng
Revision 5003 (Oct 17 2017) for Python 2.7.13 (64bit, 1thread)
Random Number Generator is set to mt19937 with random seed 0xa20f2d5453e55ba6.
This is the standard short allele version with 256 maximum allelic states.
For more information, please visit http://simupop.sourceforge.net,
or email simupop-list@lists.sourceforge.net (subscription required).


In [8]:
!jupyter nbconvert --to script simNe_functions_analyze.ipynb
import simNe_functions_analyze as simNe_anal

[NbConvertApp] Converting notebook simNe_functions_analyze.ipynb to script
[NbConvertApp] Writing 17393 bytes to simNe_functions_analyze.py


# Start

In [4]:
batchname = 'oct17a'

In [5]:
glob.glob('./share/{}/*'.format(batchname))

['./share/oct17a/Ne-200_Chr-4',
 './share/oct17a/Ne-50_Chr-4',
 './share/oct17a/Ne-800_Chr-4',
 './share/oct17a/Ne-3200_Chr-4',
 './share/oct17a/Ne-50_Chr-16',
 './share/oct17a/Ne-200_Chr-16',
 './share/oct17a/Ne-800_Chr-16',
 './share/oct17a/Ne-3200_Chr-16',
 './share/oct17a/Ne-50_Chr-64',
 './share/oct17a/Ne-200_Chr-64',
 './share/oct17a/Ne-800_Chr-64',
 './share/oct17a/Ne-3200_Chr-64']

In [6]:
target_dir = glob.glob('./share/{}/*'.format(batchname))[0]
params = pd.read_csv(os.path.join(target_dir,'params.txt'), header = None, sep = '\t')
params.columns = ['param', 'value']
params['param'] = params['param'].str.replace(' ', '')
params['value'] = params['value'].str.replace(' ', '')
dict(zip(params['param'], params['value']))

{'Ne_ancestral': '2000',
 'Ne_switchdate': '100',
 'batchname': 'oct17a',
 'chrom_len': '50000000.0',
 'coalescent_seed': '1001',
 'diploid_Ne': '200',
 'forward_seed': '1001',
 'mutation_rate': '1e-08',
 'n_chrom': '4',
 'repilcates': '500'}

In [9]:
for target_dir in glob.glob('./share/{}/*'.format(batchname)):
    print('working on dir: {}'.format(target_dir) )
    coal_file = os.path.join(target_dir,'coalescent.hdf5')
    if os.path.isfile(coal_file):
        coal = msprime.load(coal_file)
        print('successfuly loaded file: {}'.format(coal_file))
        coal = simNe_sim.filter_allele_freq(coal, MAF = 0.1)
        coal = simNe_sim.downsample_sim_data(coal, max_loci = 22000)
        # Load the params file (format will change next time):
        params = pd.read_csv(os.path.join(target_dir,'params.txt'), header = None, sep = '\t')
        params.columns = ['param', 'value']
        params['param'] = params['param'].str.replace(' ', '')
        params['value'] = params['value'].str.replace(' ', '')
        val_of_param = dict(zip(params['param'], params['value']))
        print val_of_param
        # exclude Ne == 50 for now
        #if np.int(val_of_param['diploid_Ne']) > 50:
        if len(glob.glob(os.path.join(target_dir,'*.geno'))) < np.int(val_of_param['repilcates']):
                print ('running forward simulations')      
                simNe_sim.do_forward_sims(sim_data = coal, 
                    chrom_len = np.float(val_of_param['chrom_len']), 
                    diploid_Ne = np.int(val_of_param['diploid_Ne']),
                    batchname = val_of_param['batchname'],
                    repilcates = np.int(val_of_param['repilcates']),
                    simupop_seed = np.int(val_of_param['forward_seed']))
        else:
            print ('skipping forward simulations, already done')      
    else:
        print('file {} not found, skipping'.format(coal_file))

working on dir: ./share/oct17a/Ne-200_Chr-4
successfuly loaded file: ./share/oct17a/Ne-200_Chr-4/coalescent.hdf5
34168
22000
{'batchname': 'oct17a', 'coalescent_seed': '1001', 'forward_seed': '1001', 'Ne_switchdate': '100', 'mutation_rate': '1e-08', 'chrom_len': '50000000.0', 'diploid_Ne': '200', 'repilcates': '500', 'n_chrom': '4', 'Ne_ancestral': '2000'}
skipping forward simulations, already done
working on dir: ./share/oct17a/Ne-50_Chr-4
successfuly loaded file: ./share/oct17a/Ne-50_Chr-4/coalescent.hdf5
23407
22000
{'batchname': 'oct17a', 'coalescent_seed': '1001', 'forward_seed': '1001', 'Ne_switchdate': '100', 'mutation_rate': '1e-08', 'chrom_len': '50000000.0', 'diploid_Ne': '50', 'repilcates': '500', 'n_chrom': '4', 'Ne_ancestral': '2000'}
running forward simulations
start getting chromosomes positions
done getting chromosomes positions
Exporting....1....2....3....4....5....6....7....8....9.... Done.
Exporting....1....2....3....4....5....6....7....8....9.... Done.
initalizing t

In [ ]:
assert False

In [ ]:
for target_dir in ['./share/mar14a/Ne-800_Chr-4']:
    print('working on dir: {}'.format(target_dir) )
    coal_file = os.path.join(target_dir,'coalescent.hdf5')
    if os.path.isfile(coal_file):
        coal = msprime.load(coal_file)
        print('successfuly loaded file: {}'.format(coal_file))
        simNe_sim.filter_allele_freq(coal, min_freq = .2, max_freq = .8)
        simNe_sim.downsample_sim_data(coal, max_loci = 22000)
        
        # Load the params file (format will change next time):
        params = pd.read_csv(os.path.join(target_dir,'params.txt'), header = None, sep = '\t')
        params.columns = ['param', 'value']
        params['param'] = params['param'].str.replace(' ', '')
        params['value'] = params['value'].str.replace(' ', '')
        val_of_param = dict(zip(params['param'], params['value']))
        if len(glob.glob(os.path.join(target_dir,'*.geno'))) < np.int(val_of_param['repilcates']):
            print ('running forward simulations')      
            simNe_sim.do_forward_sims(sim_data = coal, 
                chrom_len = np.float(val_of_param['chrom_len']), 
                diploid_Ne = np.int(val_of_param['diploid_Ne']),
                batchname = val_of_param['batchname'],
                repilcates = np.int(val_of_param['repilcates']),
                simupop_seed = np.int(val_of_param['forward_seed']))
        else:
            print ('skipping forward simulations, already done')      
    else:
        print('file {} not found, skipping'.format(coal_file))

In [ ]:
pass

In [ ]:
assert False

In [ ]:
        simNe_sim.do_forward_sims(sim_data = coal, 
            chrom_len = chrom_len, 
            diploid_Ne = diploid_Ne, 
            batchname = batchname,
            repilcates = repilcates,
            simupop_seed = forward_seed)

In [ ]:
# filter for freqs
        simNe_sim.filter_allele_freq(sim_data, min_freq = .2, max_freq = .8)
        simNe_sim.downsample_sim_data(sim_data, max_loci = 22000)

In [ ]:
sim_data

In [ ]:
import msprime
tt  = msprime.load('./share/mar14a/Ne-800_Chr-4/coalescent.hdf5')
tt

In [ ]:
        # do forward simualtion
        simNe_sim.do_forward_sims(sim_data = sim_data, 
            chrom_len = chrom_len, 
            diploid_Ne = diploid_Ne, 
            batchname = batchname,
            repilcates = repilcates,
            simupop_seed = forward_seed)

In [ ]:
pass

# Generate results.txt 

In [ ]:
for n_chrom in [4, 16]:
    for diploid_Ne in [200, 800, 3200]:
        try:
            simNe_anal.do_analysis(target_dir='./share/{}/Ne-{}_Chr-{}'.format(batchname, diploid_Ne, n_chrom), 
                               MAF = 0.1, 
                               L = [16, 64, 256, 1024], 
                               S = [50, 100])
        except:
            pass

In [ ]:
# Gather results
target_dir='./share/{}/Ne-{}_Chr-{}'.format(batchname, diploid_Ne, n_chrom)